We have a dataset (DCC) with a shortage of certain labels. We want to generate 
new samples synthetically using GPT-4. We will use the following approach:
1. We take the existing samples for each document type and present these to GPT-4
2. we ask to generate new sentences like it, where the token labels are provided in the BIO format

We care specifically about the following labels:
* Experiencer: Other
* Historical: Hypothetical

The task of the GPT model is to generate new sentences that are similar to the input sentences but with variations of the medical concepts. 




# Definitions
The definitions are taken from the ConText/ConTextD papers:

## Negation

This property has two values, ‘Negated’ or ‘Not negated’. A clinical condition or term is labeled as ‘Negated’ if there is evidence in the text suggesting that the condition does not occur or exist, e.g., ‘There was no sign of sinus infection’, otherwise it is ‘Not negated’.

## Temporality

The temporality property places a condition along a time line. There are three possible values for this property: ‘Recent’, ‘Historical’, and ‘Hypothetical’. A condition is considered ‘Recent’ if it is maximally 2 weeks old. Conditions that developed more than 2 weeks ago are labeled as ‘Historical’. A condition is labeled as ‘Hypothetical’ if it is not ‘Recent’ or ‘Historical’, e.g., ‘patient should return if she develops fever’ [13].

**Adaptation**: *'Hypothetical' is specifically about (theoretical) concepts, concepts that are not (yet) realized, i.e. concepts that may materialize in the future. 'Historical' and 'Recent' can be used for realized concepts, in which we also include their negations. I.e. if a concept is explicitly denied historically or recently, we can label it as 'Historical' or 'Recent' respectively.*

## Experiencer

Clinical text may refer to subjects other than the actual patient. The experiencer property describes whether the patient experienced the condition or someone else. For simplicity, we have defined only two possible values for this property: ‘Patient’ or ‘Other’, where ‘Other’ refers to anyone but the actual patient, e.g., ‘Mother is recently diagnosed with cancer’.

In [487]:
import os, sys, re
import json, dotenv
import pprint

import openai
import asyncio
from openai import AsyncOpenAI, OpenAI
from tqdm import tqdm
from collections import defaultdict

import pandas as pd

import datetime

dotenv.load_dotenv()

True

In [3]:
openai.api_key = os.getenv("OPENAI_KEY")

In [547]:
DCC = json.load(open('../data/emc-dcc_ann_Augmented.json'))

In [548]:
docs = DCC['projects'][0]['documents']

In [549]:
class_counts = {'Negation': defaultdict(int),
                'Temporality': defaultdict(int),
                'Experiencer': defaultdict(int)}

for doc in docs:
    for ann in doc['annotations']:
        for _class, val in ann['meta_anns'].items():
            class_counts[_class][val['value']] += 1

In [550]:
pprint.pprint(class_counts, indent=2)

{ 'Experiencer': defaultdict(<class 'int'>, {'patient': 12455, 'other': 886}),
  'Negation': defaultdict(<class 'int'>,
                          { 'negated': 1760,
                            'not negated': 10791}),
  'Temporality': defaultdict(<class 'int'>,
                             { 'historical': 513,
                               'hypothetical': 106,
                               'recent': 11932})}


In [551]:
relevant_docs_hypothetical = []
for i, doc in enumerate(docs):
    if doc['source'] == 'EMC_DCC_ORIGINAL':
        for concept in doc['annotations']:
            if (concept['meta_anns']['Temporality']['value']=='hypothetical'):
                doc['index'] = i
                relevant_docs_hypothetical.append(doc)
                break
        
relevant_docs_experiencer = []
for i, doc in enumerate(docs):
    if doc['source'] == 'EMC_DCC_ORIGINAL':
        for concept in doc['annotations']:
            if (concept['meta_anns']['Experiencer']['value']=='other'):
                doc['index'] = i
                relevant_docs_experiencer.append(doc)
                break

In [552]:
len(relevant_docs_hypothetical), len(relevant_docs_experiencer)

(78, 79)

In [553]:
relevant_docs_hypothetical[69]['text'][93:129]

'koorts niet gemeten.\n'

In [554]:
[(d['start'],d['end'], d['id']) for d in relevant_docs_hypothetical[69]['annotations'] 
 if d['meta_anns']['Temporality']['value']=='hypothetical']

[(93, 99, 16597)]

In [556]:
relevant_docs_hypothetical[69]['name']

'SP1753'

In [284]:
corrections = [
    {'doc_id': 'DL1616', 'annotation_id': 1873, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'DL1139', 'annotation_id': 108, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'GP2799', 'annotation_id': 8210, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'SP1476', 'annotation_id': 15532, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'DL1567', 'annotation_id': 1694, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'DL1711', 'annotation_id': 2232, 'meta': 'Temporality', 'value': 'recent'},
    {'doc_id': 'DL1812', 'annotation_id': 2232, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'DL1814', 'annotation_id': 2703, 'meta': 'Temporality', 'value': 'historical'},
 ]

In [497]:
# update the docs
update_docs = docs.copy()
for c in corrections:
    for d in update_docs:
        d['source'] = 'EMC_DCC_ORIGINAL'
        if d['name']==c['doc_id']:
            for a in d['annotations']:
                if a['id']==c['annotation_id']:
                    a['meta_anns'][c['meta']]['value'] = c['value']
# put updated docs in DCC
DCC['projects'][0]['documents'] = update_docs

In [294]:
# write DCC back to json 
with open('../data/emc-dcc_ann_NEW.json', 'w') as f:
    json.dump(DCC, f, indent=2)

In [285]:
OAI_ASYNC_CLIENT = AsyncOpenAI(api_key=os.getenv("OPENAI_KEY"), max_retries=2)
OAI_CLIENT = OpenAI(api_key=os.getenv("OPENAI_KEY"), max_retries=2)

In [492]:
SYSTEM_PROMPT_HYPOTHETICAL = """
    Je bent een kritische assistent die mij helpt om nieuwe tekst te bedenken.
    De tekst moeten voldoen aan de volgende eisen:
    - ze moeten semantisch correct zijn en vergelijkbaar zijn met de voorbeeltekst die ik je geef.
    - de voorbeeltekst wordt voorafgegaan door de term VOORBEELDTEKST
    - in de voorbeeldzin worden 1 of meer concepten benoemd die hypothethisch zijn, het is belangrijk
    dat deze concepten in de nieuwe zin ook hypothetisch zijn, het mogen ook andere concepten zijn. 
    Een voorbeeld van een hypothetische concept = 'een voorafgaand trauma kan niet worden herinnerd', waarin 'trauma' het concept is.
    Een ander voorbeeld = 'ter uitsluiting van epifysaire dysplasie' waarin 'epifysaire dysplasie' het concept is.
    - de concepten die je moet vervangen zijn aangegeven met verticale streepjes, dus |concept|.
    - het domein is medisch dus gebruik medische concepten.
    - probeer de medische concepten te varieren, dus gebruik niet steeds dezelfde concepten.
    - geef als antwoord ALLEEN de nieuw gegenereerde zinnen, voorafgaand met de term NIEUWE_TEKST
    - in de NIEUWE_TEKST, plaats de concepten die hypothetisch zijn tussen verticale streepjes, dus '|', 
    dus bijvoorbeeld: 'ter uitsluiting van |epifysaire dysplasie|'
    
In case you have doubts:
'Hypothetical' is specifically about (theoretical) concepts, which means concepts that are not (yet) realized OR
concepts that may have occurred in the past. 'Historical' and 'Recent' can be used for realized concepts, in which we also include their negations. 
I.e. if a concept is explicitly denied historically or recently, we can label it as 'Historical' or 'Recent' respectively.
"""


SYSTEM_PROMPT_EXPERIENCER = """
    Je bent een kritische assistent die mij helpt om nieuwe text te bedenken.
    Deze text moeten voldoen aan de volgende eisen:
    - het moet semantisch correct zijn en vergelijkbaar zijn met de text die ik je geef.
    - de voorbeeldtext wordt voorafgegaan door de term VOORBEELDTEKST
    - in de voorbeeldtext worden 1 of meer concepten benoemd die verwijzen naar een persoon anders dan de patient, het is belangrijk
    dat deze concepten in de nieuwe zin ook verwijzen naar iemand anders dan de patient (zoals een familielid), 
    het mogen ook andere medische concepten zijn.
    - de concepten die je moet vervangen zijn aangegeven met verticale streepjes, dus |concept|.
    - Een voorbeeld van een concept wat verwijst naar een ander persoon dan de patient =
    'Een zusje van #Name# is elders operatief behandeld in verband met recidiverende patella luxaties', waarin 'luxaties' het concept is, en er 
    wordt verwezen naar de zus van de patient.    
    - het domein is medisch dus gebruik medische concepten.
    - probeer de medische concepten te varieren, dus gebruik niet steeds dezelfde concepten.
    - varieer de ziektebeelden
    - varieer de opmaak van de text, dus gebruik niet steeds dezelfde opmaak.
    - geef als antwoord ALLEEN de nieuw gegenereerde text, voorafgaand met de term NIEUWE_TEKST
    - in de NIEUWE_TEKST, plaats alleen de concepten die verwijzen naar een ander persoon dan de patient tussen tussen verticale streepjes |, 
    dus bijvoorbeeld: 'Een zusje van #Name# is elders operatief behandeld in verband met recidiverende patella |luxaties|'
"""

In [471]:
def get_chat_res(USER_TEXT='Good day', 
                 SYSTEM_PROMPT=SYSTEM_PROMPT_HYPOTHETICAL, 
                 n = 10,
                 MODEL="gpt-4"):
    return OAI_CLIENT.chat.completions.create(
            model=MODEL,
            n = n,
            messages=[
                        {"role": "system",
                        "content": SYSTEM_PROMPT
                        },
                        {"role": "user", 
                        "content": USER_TEXT
                        }],
            stream=False,
        )

In [491]:
GPT_VERSION = 'gpt-4-1106-preview'
CURRENT_DATE = datetime.datetime.now().strftime("%Y%m%d")

In [482]:
#re_extract = re.compile(r'NIEUWE_ZIN\:(.*)')
nieuwe_zinnen_hypothetisch = []
for i, doc in tqdm(enumerate(relevant_docs_hypothetical)):
    EXAMPLE = doc['text'].replace('|', ' ')
    # add | vertical bars around the concept that needs to be replaced
    LOCS = [(d['start'],d['end']) for d in doc['annotations'] 
                if d['meta_anns']['Temporality']['value']=='hypothetical']
    for loc in LOCS:
        EXAMPLE = EXAMPLE[:loc[0]] + '|' + EXAMPLE[loc[0]:loc[1]] + '|' + EXAMPLE[loc[1]:]
    
    res = get_chat_res(SYSTEM_PROMPT=SYSTEM_PROMPT_HYPOTHETICAL, 
                       n=2,
                       MODEL=GPT_VERSION, # gpt-3.5-turbo-instruct-0914
                       USER_TEXT="VOORBEELDTEKST: " + EXAMPLE)

    for j, _res in enumerate(res.choices):
        txt = _res.message.content
        nieuwe_zinnen_hypothetisch.append((
            doc['name'],
            'hypothetisch',
            j,
            txt[txt.find('NIEUWE_TEKST')+12:].strip()))
    if i==0:
        break       
#res = pd.DataFrame(nieuwe_zinnen_hypothetisch, columns=['doc_id', 'synth_num', 'text'])
#res.to_parquet(f'../data/synth_temporality_{GPT_VERSION.replace("-", "_")}_{CURRENT_DATE}.parquet')

0it [00:10, ?it/s]


In [452]:
#re_extract = re.compile(r'NIEUWE_ZIN\:(.*)')
nieuwe_zinnen_experiencer = []
for i, doc in tqdm(enumerate(relevant_docs_experiencer)):
    EXAMPLE = doc['text'].replace('|', ' ')
    # add | vertical bars around the concept that needs to be replaced
    LOCS = [(d['start'],d['end']) for d in doc['annotations'] 
                if d['meta_anns']['Experiencer']['value']=='other']
    for loc in LOCS:
        EXAMPLE = EXAMPLE[:loc[0]] + '|' + EXAMPLE[loc[0]:loc[1]] + '|' + EXAMPLE[loc[1]:]
    
    res = get_chat_res(SYSTEM_PROMPT=SYSTEM_PROMPT_EXPERIENCER, 
                       n=10,
                       MODEL=GPT_VERSION, # gpt-3.5-turbo-instruct-0914
                       USER_TEXT="VOORBEELDTEKST: " + EXAMPLE)

    for j, _res in enumerate(res.choices):
        txt = _res.message.content
        nieuwe_zinnen_experiencer.append((
            doc['name'],
            'hypothetical',
            j,
            txt[txt.find('NIEUWE_TEKST')+12:].strip()))
        
res = pd.DataFrame(nieuwe_zinnen_experiencer, columns=['doc_id', 'synth_num', 'text'])
res.to_parquet(f'../data/synth_experiencer_{GPT_VERSION.replace("-", "_")}.parquet')

79it [20:02, 15.22s/it]


### Extract the spans from the synthetic set and add them to the original dataset with an additional label

In [539]:
for i, (name, subid, text) in enumerate(nieuwe_zinnen_experiencer):
    try:
        start, end = re.search('\|.*\|', text).span()
        start += 1
        end -= 1
        _doc = {
            'id': i,
            'source': 'synthetic',
            'source_version': f"{GPT_VERSION}|{CURRENT_DATE}",     
            'name': f"{name}|synth|{subid}",
            'text': text.replace('|', ''),
            'annotations': [
                {
                    'id': 1,
                    'user': 'emc_dcc_synth',
                    'cui': 1,
                    'start': start,
                    'end': end,
                    'value': text[start:end],
                    'validated': False,
                    'correct': True,
                    'alternative': False,
                    'killed': False,
                    'meta_anns': {
                        'Experiencer': {'value': 'other',
                                        'name': 'Experiencer',
                                        'validated': False,
                                        'acc': 1.0
                                        },
                    }            
                }
            ]
        }
    except:
        print(i, text)
    update_docs.append(_doc)

191 Geachte collega, tijdens het spreekuur op 14-02-1996 werd uw patiënt #Name# door ons gezien op de afdeling Orthopedie, doorverwezen vanuit het consultatiebureau met de vermoeden van een congenitale heupdysplasie aan de linkerzijde.
Bij het opnemen van de anamnese kwam naar voren dat #Name# vier maanden geleden geboren is via een keizersnede, volgend op een ongecompliceerde zwangerschap.
Zijn zusje had eerder te maken met een milde scoliose, ook na een keizersnede.
Aanvullend onderzoek: Röntgen van het bekken, in AP en laterale projectie: Lichte aanwijzingen voor heupdysplasie aan de linkerkant.
Conclusie: Geringe heupdysplasie aan de linkerkant.
692 Vanwege ernstige pre-eclampsie bij de moeder was een urgente keizersnede noodzakelijk voor het welzijn van de foetus.
Decursus: Momenteel is #Name# 30 maanden oud, waarbij correctie voor vroeggeboorte neerkomt op 27 maanden.
Zijn/haar ontwikkelingsquotiënt (DQ) is 89 zonder correctie en met correctie voor vroeggeboorte komt dit uit op 1

In [544]:
update_docs[-1]

{'id': 789,
 'source': 'synthetic',
 'source_version': 'gpt-4-1106-preview|20231214',
 'name': 'SP2066|synth|9',
 'text': 'AANLEIDING: onverklaarbare vermoeidheid en gewichtsverlies.\nEr is geen sprake van gastro-intestinale obstructies of maligniteiten.\nVervolgstappen: de behandelingsopties zullen worden overlegd tijdens het spreekuur endocrinologie.',
 'annotations': [{'id': 1,
   'user': 'emc_dcc_synth',
   'cui': 1,
   'start': 215,
   'end': 229,
   'value': 'endocrinologie',
   'validated': False,
   'correct': True,
   'alternative': False,
   'killed': False,
   'meta_anns': {'Experiencer': {'value': 'other',
     'name': 'Experiencer',
     'validated': False,
     'acc': 1.0}}}]}

## Write new samples to the dataset

In [545]:
DCC['projects'][0]['documents'] = update_docs

In [546]:
# write DCC back to json 
with open('../data/emc-dcc_ann_Augmented.json', 'w') as f:
    json.dump(DCC, f, indent=2)

## Translate English corpora

* English -- BioScope; [HF](https://huggingface.co/datasets/bigbio/bioscope), [src](https://rgai.inf.u-szeged.hu/downloads)
* English -- [Genia](http://www.geniaproject.org/genia-corpus/term-corpus)
* English -- Sherlock, SFU review corpus
